# Implementation

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

In [2]:
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
from packages.GramSchmidt import gram_schmidt

* **Full SVD**

$\large X_{n\times p} = U\; \Sigma\; V^* $

$U_{n\times n} =$ orthonormal eigenvectors of $XX^* $

$\Sigma_{n\times p} =$ a diagonal matrix whose diagonal entries are the square roots of the non-negative eigenvalues of both $X X^*$ and $X^* X $

${V^*}_{p\times p} =$ orthonormal eigenvectors of $X^*X$

* **Reduced(thin) SVD**

$\large X_{n\times p} = U\; S \; V^* \\
\large \displaystyle U_{n\times p} ={ \begin{pmatrix} \frac{X V_1}{s_1} &\frac{X V_2}{s_2} &\cdots & \frac{X V_p}{s_p} \end{pmatrix} } = X V S^{-1}$

$S_{p\times p} =$ a rectangular diagonal matrix whose diagonal entries are the square roots of the non-negative eigenvalues of $X^* X $

${V^*}_{p\times p} =$ orthonormal eigenvectors of $X^*X$

## Thin SVD

In [4]:
def svd_thin(data, ):
    dim = data.shape[1]
    rank = np.linalg.matrix_rank(data)
    relu = np.vectorize(lambda x: np.real(x) if np.real(x)>=0 else .0)
    
    #eval_u, evec_u = np.linalg.eig(data.dot(data.T), )
    eval_v, evec_v = np.linalg.eig(data.T.dot(data), )
    
    gs = gram_schmidt()
    #evec_u_gs = gs.fit_transform(evec_u)
    evec_v_gs = gs.fit_transform(evec_v)
        
    s = eval_v.copy()
    s = np.sqrt(relu(s))
    s1 = np.sort(s)[::-1]
    if dim > rank:
        s1[-(dim-rank):] = 0
    
    S = np.eye(dim)*s1
    
    v_idx = np.sqrt(relu(np.real(eval_v))).argsort()[-dim:][::-1]
    v = evec_v_gs[:, v_idx ]
    
    u = data.dot(v)/s1
    
    return u, S,  v

# Practice

* Sample data

$\large \mathbf{X} = {\begin{pmatrix} x_{1,1}&\cdots&x_{1,p} \\ \vdots & \ddots &\vdots \\ x_{n,1}&\cdots&x_{n,p} \end{pmatrix} }_{n\times p} = {\begin{pmatrix} X_1 & \cdots&X_p \end{pmatrix} }_{n\times p}$ where $n$ is the number of samples and $p$ is the number of features.

In [5]:
data = make_classification(n_features = 6, n_redundant=0,
                           n_samples=10**3, weights=[0.9], random_state= 42, )

In [6]:
data_x = data[0]
data_y = data[1]

The rank of the sample matrix equals to the number of columns.

In [7]:
np.linalg.matrix_rank(data_x) == data_x.shape[1]

True

$\large \displaystyle \mathbf{X}_{\normalsize standardized} = \begin{pmatrix} \frac{X_1 -  \mu_1}{\sigma_1} & \frac{X_2 -  \mu_2}{\sigma_2} & \cdots &\frac{X_p -  \mu_p}{\sigma_p}  \end{pmatrix} $
, where $\mu_i$ and $\sigma_i$ are mean and standard deviation of the vector $X_i$ for $i \in \{1,2,\cdots,p\}$.

Therefore, the means of each column of matrix $\mathbf{X}_{\normalsize standardized}$ are all 0 and standard deviations are all 1.

In [8]:
scaler = StandardScaler()
data_x_std = scaler.fit_transform(data_x)

## with numpy

In [8]:
u, s, vh = np.linalg.svd(data_x_std, full_matrices=True, )

$U$ and $V$ are all unitary matrices,

i.e. $U U^* = V V^* = I$

In [9]:
np.allclose(u.dot(u.conjugate().T), np.eye(u.shape[0]))

True

In [10]:
np.allclose(vh.dot(vh.conjugate().T), np.eye(vh.shape[0]))

True

In [11]:
S = np.zeros(data_x_std.shape, float)
np.fill_diagonal(S, s)

In [12]:
np.allclose(np.dot(u, np.dot(S, vh)), data_x_std )

True

$\large \therefore X \simeq U \Sigma V$

## my codes: reduced(thin) SVD

In [9]:
uu, ss, vv = svd_thin(data_x_std)

The matrix V whose columns consist of the right-singular vectors calculated by my codes is a unitary matrix.

i.e. $V V^* = I$

In [10]:
np.allclose((vv.conjugate().T).dot(vv), np.eye(vv.shape[0]))

True

In [11]:
np.allclose(np.dot(uu, np.dot(ss, vv.T)), data_x_std )

True

$\large \therefore X \simeq U S V$